In [ ]:
from source_files import \
    RESOLUTION, HOME_DIR, SNOW_DEPTH_DIR, CASI_COLORS, \
    aso_snow_depth, sfm_snow_depth, \
    load_aso_depth, load_sfm_depth, \
    load_hillshade, load_classifier_data, \
    load_reference_dem
from plot_helpers import *

from raster_compare.plots import PlotBase
from raster_compare.base import RasterFile

import math

from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [ ]:
aso_snow_depth_values = load_aso_depth()
sfm_snow_depth_values = load_sfm_depth(aso_snow_depth_values.mask)
hillshade = load_hillshade()

# Snow Depth Difference

In [ ]:
HIST_BIN_WIDTH = .25
bins = np.concatenate((
    np.arange(0., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sfm_snow_depth_values))],
))

In [ ]:
AREA_PLOT_OPTS = dict(
    nrows=1, ncols=2, sharex=True, sharey=True, figsize=(10,8), dpi=300
)

COLORBAR_POSITION = dict(
    right=0.90, rect=[0.91, 0.217, 0.02, 0.568],
)

SFM_UNDER = dict(color='indigo', alpha=0.4)
BLUE_CMAP.set_under(**SFM_UNDER)

imshow_opts = dict(
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=BLUE_CMAP.N,
    ),
    cmap=BLUE_CMAP,
)

In [ ]:
def area_plot():
    fig, axes = plt.subplots(**AREA_PLOT_OPTS)

    for ax in axes:
        ax.set_facecolor('whitesmoke')

        ax.imshow(
            hillshade,
            extent=sfm_snow_depth.extent,
            cmap='gray', clim=(1, 255), alpha=0.25,
        )
        ax.tick_params(axis='both', direction='inout', size=5)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
    return fig, axes

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sfm_snow_depth_values,
    extent=sfm_snow_depth.extent,
    **imshow_opts
)
    
ax1.annotate('a)', xy=(335600, 4306300), fontsize=14)
ax1.add_artist(mpatches.Circle((323000, 4319750), 250, **SFM_UNDER))
ax1.annotate('No SfM snow depth', xy=(323400, 4319950), fontsize=LABEL_SIZE)
ax1.add_artist(
    ScaleBar(1.0, location='lower left', pad=0.5, scale_loc='top', box_color='none')
)
ax1.annotate(
    'N', size=LABEL_SIZE, 
    xy=(325500, 4320050), xytext=(325500, 4322000), 
    ha="center", va="center", 
    arrowprops=dict(arrowstyle="wedge,tail_width=1.25", facecolor='black')
)

im_data = ax2.imshow(
    aso_snow_depth_values,
    extent=aso_snow_depth.extent,
    **imshow_opts,
)
ax2.annotate('b)', xy=(335600, 4306300), fontsize=14)

matplotlib.rcParams["ytick.major.pad"] = 2
PlotBase.insert_colorbar(
    ax2, 
    im_data,
    SNOW_DEPTH_LABEL,
    ticks=[0, .5, 1, 1.5, 2],
    labelpad=12,
    **COLORBAR_POSITION
)

# Insert zoom figure
ax2.add_patch(
    Rectangle((327200, 4320470 - 13879), 1000, 1000,  ec='darkred', ls='--', fill=False),
)
ax2.legend(handles=[
    mlines.Line2D([0], [0], label='Figure 4', color='darkred', ls='--'),
], loc='lower left', facecolor='none', edgecolor='none')

del im_data

## Zoom comparison

In [ ]:
rgb_zoom_tif = plt.imread((HOME_DIR / 'ERW_zoom_RGB/ERW_zoom.tif').as_posix())

casi_zoom = RasterFile(
    SNOW_DEPTH_DIR / f'20180524_ASO_CASI_ERW_basin_{RESOLUTION}_zoom.vrt',
    band_number=1
)
casi_zoom_values = casi_zoom.band_values();

In [ ]:
matplotlib.rcParams['axes.titlesize'] = 8
matplotlib.rcParams['axes.titlepad'] = 2
matplotlib.rcParams['axes.labelsize'] = 8
matplotlib.rcParams['axes.labelpad'] = 0
matplotlib.rcParams['xtick.labelsize'] = 0
matplotlib.rcParams['ytick.labelsize'] = 0

In [ ]:
def plot_zoom(resolution, ax1, ax2):
    ax1.set_facecolor('whitesmoke')
    ax2.set_facecolor('whitesmoke')
    
    aso_snow_depth_zoom = RasterFile(
        HOME_DIR / f'{resolution}m/20180524_ASO_snow_depth_{resolution}m_zoom.vrt',
        band_number=1
    )
    aso_snow_depth_zoom_values = aso_snow_depth_zoom.band_values()
    aso_snow_depth_zoom_values = np.ma.masked_where(
        aso_snow_depth_zoom_values <= 0.0,
        aso_snow_depth_zoom_values,
        copy=False
    )

    sfm_snow_depth_zoom = RasterFile(
        HOME_DIR / f'{resolution}m/20180524_Agisoft_snow_depth_{resolution}m_zoom.vrt',
        band_number=1
    )
    sfm_snow_depth_zoom_values = np.ma.masked_where(
        aso_snow_depth_zoom_values.mask,
        sfm_snow_depth_zoom.band_values(),
        copy=False
    )
    
    ax1.imshow(
        sfm_snow_depth_zoom_values,
        extent=sfm_snow_depth_zoom.extent,
        **imshow_opts
    )
    ax1.set_ylabel(f' {resolution}m Resolution')
    
    return ax2.imshow(
        aso_snow_depth_zoom_values,
        extent=aso_snow_depth_zoom.extent,
        **imshow_opts
    )


In [ ]:
fig = plt.figure(dpi=300)
grid_spec = fig.add_gridspec(
    nrows=1, ncols=2, wspace=0.05, width_ratios=[0.6, 0.3]
)

cell_grid = grid_spec[0].subgridspec(2, 2, wspace=0.02, hspace=.1)
ax_1_1 = fig.add_subplot(cell_grid[0, 0])
ax_1_2 = fig.add_subplot(cell_grid[0, 1:])
ax_2_1 = fig.add_subplot(cell_grid[1, 0])
ax_2_2 = fig.add_subplot(cell_grid[1, 1:])

im_data = plot_zoom(3, ax_1_1, ax_1_2)
ax_1_1.set_title('SfM')
ax_1_2.set_title('ASO')

plot_zoom(50, ax_2_1, ax_2_2)

cell_grid = grid_spec[1].subgridspec(2, 1, wspace=0.02, hspace=.1)
ax1 = fig.add_subplot(cell_grid[0, 0])
ax2 = fig.add_subplot(cell_grid[1, 0])

ax1.imshow(
    casi_zoom_values, 
    extent=casi_zoom.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax1.set_title('Classification')

ax2.imshow(
    rgb_zoom_tif,
    extent=casi_zoom.extent,
)
ax2.set_title('Orthomosaic', pad=-4)

for ax in fig.get_axes():
    ax.tick_params(axis='both', direction='inout', size=5)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

## CASI Classification 

In [ ]:
classifier_data = load_classifier_data(aso_snow_depth_values.mask)
sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    sd_difference_values,
    copy=False
)
classification_plot = np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    classifier_data,
).astype(np.int8);

In [ ]:
bins = np.concatenate((
    [math.floor(np.nanmin(sd_difference_values))],
    np.arange(-2., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sd_difference_values))],
))
imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=RED_BLUE_CMAP.N,
    ),
    cmap=RED_BLUE_CMAP,
)

In [ ]:
fig, (ax1, ax2) = area_plot()
fig.subplots_adjust(wspace=.15)

im_data = ax1.imshow(
    sd_difference_values,     
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")
PlotBase.insert_colorbar(ax1, im_data, 'Snow Depth Difference (m)')

im_data = ax2.imshow(
    classification_plot, 
    extent=sfm_snow_depth.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax2.set_title("Snow Depth Differences - Classification")
PlotBase.insert_colorbar(ax2, im_data, 'Classification');

In [ ]:
dem, dem_values = load_reference_dem(aso_snow_depth_values.mask)

In [ ]:
high_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3100, 3500).mask,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3800, 3900).mask,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()
ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - 3100m < Elevation < 3200m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - 3800 < Elevation < 3900")

PlotBase.insert_colorbar(ax2, im_data, r'$\Delta$ Snow Depth (m)', **COLORBAR_POSITION);

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sd_difference_values, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")

im_data = ax2.imshow(
    sfm_snow_free_values - dem_values, 
    **imshow_opts,
)
ax2.set_title("Snow Free - Reference DEM")

PlotBase.insert_colorbar(ax2, im_data, **COLORBAR_POSITION);

In [ ]:
data = [
    {
        'data': sfm_snow_free_values - dem_values,
        'label': 'SfM snow free - DEM',
        'color': 'dodgerblue',
    },
]

with plot_histogram(data, (-6, 6), figsize=(10, 8)) as ax:
    ax.set_title('Elevation Differences (SFM snow free - Reference DEM)');